# Importing data from GLDAS nc files

#### Necessary libraries

In [ ]:
import pandas as pd
from netCDF4 import Dataset
import glob
import re

#### Coordinates of ground stations

In [ ]:
input_latlon = {35.818:51.781,
                35.864:51.658,
                35.896:51.6,
                35.935:51.434,
                35.953:51.364,
                35.985:51.639,
                35.999:50.983,
                36.026:51.513,
                36.034:51.423,
                36.064:51.126,
                36.08:51.324,
                36.105:51.311,
                36.14:50.688,
                36.146:50.749,
                36.177:50.909,
                36.195:50.898,
                36.21:50.767,
                36.226:50.956}

#### Main body of code

In [ ]:
time_table = []
lat_table = []
lon_table = []
sw_table = []


# Change it to nc files's folder
for file in glob.glob('F:/GLDAS/*.nc4'):

    for key in input_latlon:
        
        # Extract time from file names. (\d+) is where the time is.
        #Change before and after of (\d+), according to file names.
        res = re.findall("GLDAS_CLSM025_DA1_D.A(\d+).022.nc4.SUB",file)
        
        time_table.append(res[0])
    
        ds = Dataset(file, 'r')
        
        ds_lat = ds['lat'][:]
        ds_lon = ds['lon'][:]
        
        #Change it to name of variable you want to save.
        #Check inside of nc file for the available variables.
        SWE = ds['SWE_tavg'][:]
        
        
        lat_table.append(key)
        lon_table.append(input_latlon[key])
        
        sq_diff_lat = (ds_lat - key)**2
        sq_diff_lon = (ds_lon - input_latlon[key])**2
        
        min_index_lat = sq_diff_lat.argmin()
        min_index_lon = sq_diff_lon.argmin()
        
        sw = SWE[0, min_index_lat ,min_index_lon]
        sw_table.append(sw)

#### Merge all data to a DataFrame

In [ ]:
all_table = {'Time':time_table[:],
             'Lat':lat_table[:],
             'Lon':lon_table[:],
            'SWE':sw_table[:]}

final_ds = pd.DataFrame(all_table)
final_ds["Time"] = pd.to_datetime(final_ds["Time"])

#### Export DataFrame to a Excel file

In [ ]:
final_ds.to_excel("f:/gldas_ds.xlsx",index=False)